In [36]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

url='https://en.wikipedia.org/wiki/2025%E2%80%9326_Premier_League'

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

page=requests.get(url,headers=headers)

soup=BeautifulSoup(page.text,'lxml')

table=soup.find_all('table',class_='wikitable')[3]



head=table.find_all('th',scope='col')

head_table=[title.text for title in head]
columns=[head.strip() for head in head_table]


df=pd.DataFrame(columns=columns)
df


,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts,Qualification or relegation


In [37]:
rows=table.find_all('tr')

for row in rows[1:]:
    row_header=row.find('th',scope='row')
    team=row_header.text.strip() if row_header else ""

    cells=[td.text.strip() for td in row.find_all('td')]

    row_data=[team]+cells

    if len(row_data)<len(columns):
        row_data+=['']*(len(columns)-len(row_data))

    df.loc[len(df)]=row_data


df.head()






,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts,Qualification or relegation
0,Arsenal,1,16,11,3,2,30,10,+20,36,Qualification for the Champions League league ...
1,Manchester City,2,16,11,1,4,38,16,+22,34,
2,Aston Villa,3,16,10,3,3,25,17,+8,33,
3,Chelsea,4,16,8,4,4,27,15,+12,28,
4,Crystal Palace,5,16,7,5,4,20,15,+5,26,Qualification for the Europa League league pha...


## dropped the qualification or relegation columns since we do not need it to make predictions


In [38]:
df.drop(columns='Qualification or relegation',inplace=True)
df

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,Arsenal,1,16,11,3,2,30,10,+20,36
1,Manchester City,2,16,11,1,4,38,16,+22,34
2,Aston Villa,3,16,10,3,3,25,17,+8,33
3,Chelsea,4,16,8,4,4,27,15,+12,28
4,Crystal Palace,5,16,7,5,4,20,15,+5,26
5,Manchester United,6,16,7,5,4,30,26,+4,26
6,Liverpool,7,16,8,2,6,26,24,+2,26
7,Sunderland,8,16,7,5,4,19,17,+2,26
8,Everton,9,16,7,3,6,18,19,−1,24
9,Brighton & Hove Albion,10,16,6,5,5,25,23,+2,23


In [39]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


features=['Pld','W','D','L','GF','GA','GD']
target='Pts'

df.rename(columns={'Team':'Pos',"Pos":"Team"},inplace=True)
df






,Team,Pos,Pld,W,D,L,GF,GA,GD,Pts
0,Arsenal,1,16,11,3,2,30,10,+20,36
1,Manchester City,2,16,11,1,4,38,16,+22,34
2,Aston Villa,3,16,10,3,3,25,17,+8,33
3,Chelsea,4,16,8,4,4,27,15,+12,28
4,Crystal Palace,5,16,7,5,4,20,15,+5,26
5,Manchester United,6,16,7,5,4,30,26,+4,26
6,Liverpool,7,16,8,2,6,26,24,+2,26
7,Sunderland,8,16,7,5,4,19,17,+2,26
8,Everton,9,16,7,3,6,18,19,−1,24
9,Brighton & Hove Albion,10,16,6,5,5,25,23,+2,23


In [53]:
# df[['Pts','Pos','Pld','W','D','L','GA','GF']]=df[['Pts','Pos','Pld','W','D','L','GA','GF']].astype(int)
for col in ['Pts','Pos','Pld','W','D','L','GA','GF']:
    df[col]=df[col].astype(int)

df['GD']=df['GD'].astype('str')
df['GD'] = df['GD'].str.replace('−', '-', regex=False)
df['GD'] = df['GD'].str.replace('+', '', regex=False)

df['GD']=pd.to_numeric(df['GD'])

df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 20 entries, 0 to 19
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Team    20 non-null     object
 1   Pos     20 non-null     int32 
 2   Pld     20 non-null     int32 
 3   W       20 non-null     int32 
 4   D       20 non-null     int32 
 5   L       20 non-null     int32 
 6   GF      20 non-null     int32 
 7   GA      20 non-null     int32 
 8   GD      20 non-null     int64 
 9   Pts     20 non-null     int32 
dtypes: int32(8), int64(1), object(1)
memory usage: 1.1+ KB


In [54]:
df.head()

,Team,Pos,Pld,W,D,L,GF,GA,GD,Pts
0,Arsenal,1,16,11,3,2,30,10,20,36
1,Manchester City,2,16,11,1,4,38,16,22,34
2,Aston Villa,3,16,10,3,3,25,17,8,33
3,Chelsea,4,16,8,4,4,27,15,12,28
4,Crystal Palace,5,16,7,5,4,20,15,5,26
